# Imports Data Cleaning Notebook

This notebook performs comprehensive data cleaning on the ABS imports dataset for 2024-2025.

## Cleaning Steps:
1. Load and inspect the raw data
2. Handle missing values
3. Convert data types
4. Standardize text fields
5. Handle outliers and invalid values
6. Create derived features
7. Remove duplicates
8. Save cleaned dataset


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


Libraries imported successfully!
Pandas version: 2.1.4
NumPy version: 1.26.4


## 1. Load and Inspect Raw Data


In [2]:
# Load the raw imports data
data_path = 'data/imports_2024_2025.csv'

df = pd.read_csv(data_path)
print(f"\n Data loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names:")
print(df.columns.tolist())



 Data loaded successfully!
Shape: 4,481,579 rows × 18 columns

Column names:
['month', 'mode', 'mode_description', 'commodity_code', 'commodity_description', 'ausport_code', 'ausport_description', 'osport_code', 'osport_description', 'state', 'country_code', 'country_description', 'weight', 'valuefob', 'valuecif', 'unit_quantity', 'quantity', 'year']


In [3]:
# Display basic information about the dataset
print("=== DATASET OVERVIEW ===")
df.head()


=== DATASET OVERVIEW ===


,month,mode,mode_description,commodity_code,commodity_description,ausport_code,ausport_description,osport_code,osport_description,state,country_code,country_description,weight,valuefob,valuecif,unit_quantity,quantity,year
0,January 2024,A,AIR,79391,"Rafts, inflatable",101,Sydney,840460,Honolulu,New South Wales,USA,United States of America,0.0,1204.37,1244.67,Number,2.00,2024
1,January 2024,S,SEA,7528,Saffron,101,Sydney,840468,Los Angeles,New South Wales,SPAI,Spain,0.0,58646.15,60840.25,Kilograms,416.40,2024
2,January 2024,A,AIR,69940,"Springs and leaves for springs, of iron or steel",101,Sydney,276150,Frankfurt,Victoria,FGMY,Germany,0.0,4997.11,5094.51,Number,1204.00,2024
3,January 2024,A,AIR,74831,Articulated link roller chain of iron or steel,523,Perth,528050,Amsterdam,Western Australia,JAP,Japan,0.0,3343.30,3468.06,Kilograms,103.73,2024
4,January 2024,S,SEA,82116,"Wooden framed seats, nes",401,Port Adelaide,276450,Hamburg,South Australia,FGMY,Germany,0.0,15122.99,15779.34,Number,43.00,2024


In [4]:
# Check data types and missing values
print("=== DATA TYPES & MISSING VALUES ===")
print("\nData types:")
print(df.dtypes)
print("\n\nMissing values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found!")


=== DATA TYPES & MISSING VALUES ===

Data types:
month                     object
mode                      object
mode_description          object
commodity_code             int64
commodity_description     object
ausport_code               int64
ausport_description       object
osport_code                int64
osport_description        object
state                     object
country_code              object
country_description       object
weight                   float64
valuefob                 float64
valuecif                 float64
unit_quantity             object
quantity                 float64
year                       int64
dtype: object


Missing values:
No missing values found!


In [5]:
# Check for duplicates
print("=== DUPLICATE CHECK ===")
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count:,}")
if duplicate_count > 0:
    print(f"Percentage: {(duplicate_count / len(df)) * 100:.2f}%")


=== DUPLICATE CHECK ===
Duplicate rows: 0


## 2. Data Cleaning and Conversion


In [6]:
# Create a copy for cleaning
df_clean = df.copy()
print("=== STARTING DATA CLEANING ===\n")


=== STARTING DATA CLEANING ===



In [7]:
# Standardize column names (convert to lowercase with underscores for consistency)
print("Standardizing column names...")
df_clean.columns = df_clean.columns.str.lower().str.replace(' ', '_')
print(f"Column names standardized")
print(f"New columns: {df_clean.columns.tolist()}")


Standardizing column names...
Column names standardized
New columns: ['month', 'mode', 'mode_description', 'commodity_code', 'commodity_description', 'ausport_code', 'ausport_description', 'osport_code', 'osport_description', 'state', 'country_code', 'country_description', 'weight', 'valuefob', 'valuecif', 'unit_quantity', 'quantity', 'year']


In [8]:
# Convert numeric columns and handle missing values
print("\n=== CONVERTING NUMERIC COLUMNS ===")

# Numeric columns in imports data
numeric_columns = ['weight', 'valuefob', 'valuecif', 'quantity']

for col in numeric_columns:
    if col in df_clean.columns:
        print(f"\nProcessing {col}...")
        
        # Convert to numeric
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        # Check missing values
        missing_count = df_clean[col].isnull().sum()
        if missing_count > 0:
            print(f"  Missing values: {missing_count:,} ({(missing_count/len(df_clean))*100:.2f}%)")
            # Fill missing with 0 for trade data
            df_clean[col] = df_clean[col].fillna(0)
        
        # Handle negative values (shouldn't exist in trade data)
        negative_count = (df_clean[col] < 0).sum()
        if negative_count > 0:
            print(f"  Negative values found: {negative_count:,}")
            df_clean[col] = df_clean[col].clip(lower=0)
        
        # Show summary statistics
        print(f"  Min: {df_clean[col].min():,.2f}, Max: {df_clean[col].max():,.2f}, Mean: {df_clean[col].mean():,.2f}")

print("\n Numeric columns processed")



=== CONVERTING NUMERIC COLUMNS ===

Processing weight...
  Negative values found: 22,151
  Min: 0.00, Max: 530,451.00, Mean: 44.62

Processing valuefob...
  Negative values found: 36,030
  Min: 0.00, Max: 2,923,069,554.39, Mean: 178,885.61

Processing valuecif...
  Negative values found: 36,030
  Min: 0.00, Max: 2,939,111,352.40, Mean: 186,456.17

Processing quantity...
  Min: 0.00, Max: 789,189,365.00, Mean: 44,881.26

 Numeric columns processed


In [9]:
# Clean text columns
print("\n=== CLEANING TEXT COLUMNS ===")

# Text columns that should be standardized
text_columns = ['country_description', 'commodity_description', 'mode_description', 
                'ausport_description', 'osport_description', 'state', 'unit_quantity']

for col in text_columns:
    if col in df_clean.columns:
        print(f"\nProcessing {col}...")
        
        # Convert to string and handle missing values
        df_clean[col] = df_clean[col].astype(str)
        
        # Replace 'nan' strings and actual NaN with 'Unknown'
        df_clean[col] = df_clean[col].replace({'nan': 'Unknown', 'NaN': 'Unknown', 'None': 'Unknown'})
        df_clean[col] = df_clean[col].fillna('Unknown')
        
        # Clean whitespace
        df_clean[col] = df_clean[col].str.strip()
        df_clean[col] = df_clean[col].str.replace(r'\s+', ' ', regex=True)  # Multiple spaces to single
        df_clean[col] = df_clean[col].str.replace(r'[\n\r\t]+', ' ', regex=True)  # Remove newlines/tabs
        
        # Remove quotes
        df_clean[col] = df_clean[col].str.replace('"', '', regex=False)
        df_clean[col] = df_clean[col].str.replace("'", '', regex=False)
        
        # Count unique values
        unique_count = df_clean[col].nunique()
        print(f"  Unique values: {unique_count:,}")

print("\nText columns cleaned")



=== CLEANING TEXT COLUMNS ===

Processing country_description...
  Unique values: 224

Processing commodity_description...
  Unique values: 2,897

Processing mode_description...
  Unique values: 4

Processing ausport_description...
  Unique values: 61

Processing osport_description...
  Unique values: 590

Processing state...
  Unique values: 9

Processing unit_quantity...
  Unique values: 16

Text columns cleaned


## 2.5. Unit Validation and Standardization


In [10]:
# Standardize unit_quantity names
print("\n=== STANDARDIZING UNIT NAMES ===")

# Create unit standardization mapping
unit_mapping = {
    'Litres Al': 'Litres',  # Standardize variation
    'litres al': 'Litres',
    'LITRES AL': 'Litres',
    # Add other variations if found
}

# Check for other potential variations
print("Checking for unit name variations...")
original_units = df_clean['unit_quantity'].value_counts()
print(f"Unique unit types before standardization: {len(original_units)}")

# Apply standardization
before_count = len(df_clean)
df_clean['unit_quantity_original'] = df_clean['unit_quantity'].copy()  # Keep original for reference
df_clean['unit_quantity'] = df_clean['unit_quantity'].map(unit_mapping).fillna(df_clean['unit_quantity'])

# Check what changed
changed = (df_clean['unit_quantity_original'] != df_clean['unit_quantity']).sum()
if changed > 0:
    print(f" Standardized {changed:,} unit names")
    print("\nChanges made:")
    changes = df_clean[df_clean['unit_quantity_original'] != df_clean['unit_quantity']][['unit_quantity_original', 'unit_quantity']].value_counts()
    print(changes)
else:
    print("No unit name variations found to standardize")

print(f"\nUnique unit types after standardization: {df_clean['unit_quantity'].nunique()}")




=== STANDARDIZING UNIT NAMES ===
Checking for unit name variations...
Unique unit types before standardization: 16
 Standardized 11,691 unit names

Changes made:
unit_quantity_original  unit_quantity
Litres Al               Litres           11691
Name: count, dtype: int64

Unique unit types after standardization: 15


In [11]:
# Round "Number" unit quantities to integers
print("\n=== ROUNDING NUMBER UNIT QUANTITIES ===")

# Find Number units with decimals
number_mask = df_clean['unit_quantity'] == 'Number'
if number_mask.any():
    number_data = df_clean[number_mask].copy()
    quantities = pd.to_numeric(number_data['quantity'], errors='coerce')
    
    # Find records with decimals
    has_decimals = (quantities % 1 != 0) & (~quantities.isna())
    decimal_count = has_decimals.sum()
    
    if decimal_count > 0:
        print(f"Found {decimal_count:,} 'Number' records with decimal quantities")
        print(f"  Percentage: {(decimal_count/number_mask.sum())*100:.2f}%")
        
        # Show examples before rounding
        print("\n  Examples before rounding:")
        examples = df_clean.loc[number_mask & has_decimals, ['unit_quantity', 'quantity', 'commodity_description']].head(5)
        print(examples.to_string(index=False))
        
        # Round to nearest integer
        df_clean.loc[number_mask, 'quantity'] = pd.to_numeric(df_clean.loc[number_mask, 'quantity'], errors='coerce').round(0)
        
        print(f"\n Rounded {decimal_count:,} 'Number' quantities to integers")
        
        # Verify
        quantities_after = pd.to_numeric(df_clean.loc[number_mask, 'quantity'], errors='coerce')
        still_decimal = (quantities_after % 1 != 0).sum()
        if still_decimal == 0:
            print("Verification: All 'Number' quantities are now integers")
        else:
            print(f" Warning: {still_decimal} 'Number' quantities still have decimals")
    else:
        print("No 'Number' units with decimals found")
else:
    print("No 'Number' units found in dataset")




=== ROUNDING NUMBER UNIT QUANTITIES ===
Found 6,353 'Number' records with decimal quantities
  Percentage: 0.34%

  Examples before rounding:
unit_quantity  quantity                               commodity_description
       Number   4142.32                                   Base metal hinges
       Number 248633.95                                Razors, non-electric
       Number    157.66  Intake air filters for internal combustion engines
       Number    118.46 Valves for oleohydraulic or pneumatic transmissions
       Number  11660.41                                    Printed circuits

 Rounded 6,353 'Number' quantities to integers
Verification: All 'Number' quantities are now integers


In [12]:
# Validate weight units (assuming tonnes, just verify consistency)
print("\n=== WEIGHT UNIT VALIDATION ===")

weights = pd.to_numeric(df_clean['weight'], errors='coerce')

print(f"Weight statistics:")
print(f"  Min: {weights.min():,.2f} tonnes")
print(f"  Max: {weights.max():,.2f} tonnes")
print(f"  Mean: {weights.mean():,.2f} tonnes")
print(f"  Median: {weights.median():,.2f} tonnes")

# Check distribution
small_weights = (weights < 1).sum()
very_small_weights = (weights < 0.1).sum()
large_weights = (weights > 1000).sum()

print(f"\nWeight distribution:")
print(f"  Weights < 1 tonne: {small_weights:,} ({(small_weights/len(df_clean))*100:.2f}%)")
print(f"  Weights < 0.1 tonne: {very_small_weights:,} ({(very_small_weights/len(df_clean))*100:.2f}%)")
print(f"  Weights > 1000 tonnes: {large_weights:,} ({(large_weights/len(df_clean))*100:.2f}%)")

print("\n Weight validation completed (assuming all weights are in tonnes)")
print("   Note: Small weights (< 1 tonne) are normal for small shipments")



=== WEIGHT UNIT VALIDATION ===
Weight statistics:
  Min: 0.00 tonnes
  Max: 530,451.00 tonnes
  Mean: 44.62 tonnes
  Median: 0.01 tonnes

Weight distribution:
  Weights < 1 tonne: 3,439,041 (76.74%)
  Weights < 0.1 tonne: 2,840,135 (63.37%)
  Weights > 1000 tonnes: 13,669 (0.31%)

 Weight validation completed (assuming all weights are in tonnes)
   Note: Small weights (< 1 tonne) are normal for small shipments


In [13]:
# Clean month column - extract month name only (remove year since year column exists)
print("\n=== PROCESSING DATE INFORMATION ===")

if 'month' in df_clean.columns:
    # Ensure month is string
    df_clean['month'] = df_clean['month'].astype(str)
    
    # Extract month name only (remove year part) since year column already exists
    # Extract just the month name from "January 2024" -> "January"
    month_name = df_clean['month'].str.extract(r'^([A-Za-z]+)')[0]
    df_clean['month'] = month_name  # Replace month column with just month name
    
    # Create month number for analysis
    month_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4,
        'May': 5, 'June': 6, 'July': 7, 'August': 8,
        'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    
    df_clean['month_number'] = df_clean['month'].map(month_map)
    
    # Verify year column exists (should already be present)
    if 'year' in df_clean.columns:
        print(f"  Years in dataset: {sorted(df_clean['year'].dropna().unique().tolist())}")
    else:
        print("  ⚠️  Warning: Year column not found")
    
    print(f"  Month column now contains: {df_clean['month'].unique()[:5].tolist()}...")
    print(f"  Month numbers range: {df_clean['month_number'].min()} to {df_clean['month_number'].max()}")

print("\n✅ Date information processed")



=== PROCESSING DATE INFORMATION ===
  Years in dataset: [2024, 2025]
  Month column now contains: ['January', 'February', 'May', 'March', 'April']...
  Month numbers range: 1 to 12

✅ Date information processed


In [14]:
# Create derived features
print("\n=== CREATING DERIVED FEATURES ===")

# Calculate value per tonne (using weight)
if 'valuefob' in df_clean.columns and 'weight' in df_clean.columns:
    # Convert weight to tonnes if needed (assuming weight is in tonnes already)
    denominator = df_clean['weight'].replace(0, np.nan)
    df_clean['value_per_tonne_fob'] = df_clean['valuefob'] / denominator
    print("  Created: value_per_tonne_fob")

if 'valuecif' in df_clean.columns and 'weight' in df_clean.columns:
    denominator = df_clean['weight'].replace(0, np.nan)
    df_clean['value_per_tonne_cif'] = df_clean['valuecif'] / denominator
    print("  Created: value_per_tonne_cif")

# Calculate difference between CIF and FOB (insurance and freight costs)
if 'valuecif' in df_clean.columns and 'valuefob' in df_clean.columns:
    df_clean['insurance_freight_cost'] = df_clean['valuecif'] - df_clean['valuefob']
    df_clean['insurance_freight_cost'] = df_clean['insurance_freight_cost'].clip(lower=0)  # Shouldn't be negative
    print("  Created: insurance_freight_cost")

# Add processing date
#df_clean['data_processed_date'] = datetime.now().strftime('%Y-%m-%d')
#print("  Created: data_processed_date")

print("\nDerived features created")



=== CREATING DERIVED FEATURES ===
  Created: value_per_tonne_fob
  Created: value_per_tonne_cif
  Created: insurance_freight_cost

Derived features created


In [15]:
# Remove duplicates
print("\n=== REMOVING DUPLICATES ===")
before_count = len(df_clean)
df_clean = df_clean.drop_duplicates()
after_count = len(df_clean)
removed = before_count - after_count

print(f"  Before: {before_count:,} rows")
print(f"  After: {after_count:,} rows")
print(f"  Removed: {removed:,} duplicate rows ({(removed/before_count)*100:.2f}%)")

print("\n Duplicates removed")



=== REMOVING DUPLICATES ===
  Before: 4,481,579 rows
  After: 4,481,579 rows
  Removed: 0 duplicate rows (0.00%)

 Duplicates removed


## 3. Data Quality Summary


In [16]:
# Final data quality check
print("=== FINAL DATA QUALITY SUMMARY ===\n")

print(f"Total rows: {len(df_clean):,}")
print(f"Total columns: {len(df_clean.columns)}")
print(f"\nData types:")
print(df_clean.dtypes.value_counts())

print(f"\nMissing values (after cleaning):")
final_missing = df_clean.isnull().sum()
final_missing_df = pd.DataFrame({
    'Missing Count': final_missing,
    'Percentage': (final_missing / len(df_clean)) * 100
})
final_missing_df = final_missing_df[final_missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(final_missing_df) > 0:
    print(final_missing_df)
else:
    print("✅ No missing values!")

print(f"\nSummary statistics for numeric columns:")
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
if numeric_cols:
    print(df_clean[numeric_cols].describe())


=== FINAL DATA QUALITY SUMMARY ===

Total rows: 4,481,579
Total columns: 23

Data types:
object     11
float64     7
int64       5
Name: count, dtype: int64

Missing values (after cleaning):
                     Missing Count  Percentage
value_per_tonne_fob        1823199   40.682068
value_per_tonne_cif        1823199   40.682068

Summary statistics for numeric columns:
       commodity_code  ausport_code   osport_code        weight      valuefob  \
count    4.481579e+06  4.481579e+06  4.481579e+06  4.481579e+06  4.481579e+06   
mean     7.137245e+04  2.463182e+02  4.889941e+05  4.461985e+01  1.788856e+05   
std      1.896213e+04  1.965422e+02  2.711864e+05  1.471221e+03  3.747555e+06   
min      1.500000e+02  1.010000e+02  4.399000e+03  0.000000e+00  0.000000e+00   
25%      6.661300e+04  1.010000e+02  2.504540e+05  0.000000e+00  2.711180e+03   
50%      7.478000e+04  2.010000e+02  4.580110e+05  9.500000e-03  9.085290e+03   
75%      8.311200e+04  3.010000e+02  7.640030e+05  6.970000e

In [17]:
# Show breakdown by key dimensions
print("\n=== DATA BREAKDOWN ===")

if 'year' in df_clean.columns:
    print("\nRecords by year:")
    year_counts = df_clean['year'].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count:,} records")

if 'mode_description' in df_clean.columns:
    print("\nRecords by transport mode:")
    mode_counts = df_clean['mode_description'].value_counts()
    for mode, count in mode_counts.items():
        print(f"  {mode}: {count:,} records")

if 'country_description' in df_clean.columns:
    print("\nTop 10 countries by record count:")
    country_counts = df_clean['country_description'].value_counts().head(10)
    for country, count in country_counts.items():
        print(f"  {country}: {count:,} records")



=== DATA BREAKDOWN ===

Records by year:
  2024: 2,684,331 records
  2025: 1,797,248 records

Records by transport mode:
  SEA: 2,283,382 records
  AIR: 2,193,846 records
  POST: 4,311 records
  MODE OF TRANSPORT NOT AVAILABLE FOR PUBLICATION: 40 records

Top 10 countries by record count:
  China (excludes SARs and Taiwan): 931,539 records
  United States of America: 536,656 records
  Germany: 310,477 records
  Italy (includes Holy See and San Marino): 237,254 records
  India: 174,615 records
  Japan: 173,761 records
  United Kingdom, Channel Islands and Isle of Man, nfd: 170,358 records
  France (includes Andorra and Monaco): 127,275 records
  Taiwan: 119,187 records
  Vietnam: 103,872 records


## 4. Save Cleaned Dataset


In [18]:
# Save cleaned dataset
output_path = 'data/imports_2024_2025_cleaned.csv'
print(f"Saving cleaned data to: {output_path}")

df_clean.to_csv(output_path, index=False)
print(f" Cleaned dataset saved successfully!")
print(f"   File size: {len(df_clean):,} rows × {len(df_clean.columns)} columns")


Saving cleaned data to: data/imports_2024_2025_cleaned.csv
 Cleaned dataset saved successfully!
   File size: 4,481,579 rows × 23 columns


In [19]:
# Display sample of cleaned data
print("\n=== SAMPLE OF CLEANED DATA ===\n")
sample_cols = ['month', 'year', 'country_description', 'commodity_description', 
               'mode_description', 'valuefob', 'valuecif', 'weight', 'quantity']
available_cols = [col for col in sample_cols if col in df_clean.columns]
df_clean[available_cols].head(10)



=== SAMPLE OF CLEANED DATA ===



,month,year,country_description,commodity_description,mode_description,valuefob,valuecif,weight,quantity
0,January,2024,United States of America,"Rafts, inflatable",AIR,1204.37,1244.67,0.00000,2.00
1,January,2024,Spain,Saffron,SEA,58646.15,60840.25,0.00000,416.40
2,January,2024,Germany,"Springs and leaves for springs, of iron or steel",AIR,4997.11,5094.51,0.00000,1204.00
3,January,2024,Japan,Articulated link roller chain of iron or steel,AIR,3343.30,3468.06,0.00000,103.73
4,January,2024,Germany,"Wooden framed seats, nes",SEA,15122.99,15779.34,0.00000,43.00
5,February,2024,China (excludes SARs and Taiwan),Soya sauce,SEA,29673.94,31669.19,24.18000,18600.00
6,February,2024,Taiwan,"Safety headgear, whether or not lined or trimmed",SEA,151920.02,156097.23,2.29230,6616.00
7,February,2024,Taiwan,"Food preparations, not elsewhere specified",SEA,557493.60,583064.65,120.75764,126936.14
8,February,2024,Spain,Seats of a kind used for motor vehicles,AIR,3144.84,3712.08,0.00000,1.00
9,February,2024,Myanmar,"Footwear with outer soles of leather, nes",SEA,3455.28,3602.60,0.00000,38.00


## Summary

**Data cleaning completed successfully!**

### What was done:
-  Standardized column names
-  Converted numeric columns and handled missing/negative values
-  Cleaned and standardized text fields
-  Extracted date components (year, month number)
-  Created derived features (value per tonne, insurance/freight costs)
-  Removed duplicate records
- Saved cleaned dataset to CSV format

